# Exercise: Transition-based dependency parsing

In this exercise we will implement and evaluate the implementation of a simple transition-based dependency parser in the style of [Chen and Manning (2014)](https://www.aclweb.org/anthology/D14-1082/). This parser is based on the arc-standard algorithm that was presented in the video lectures.

## Dataset

The dataset for the exercise is the English Web Treebank from the [Universal Dependencies Project](http://universaldependencies.org), a corpus containing more than 16,000 sentences (254,000&nbsp;tokens) annotated with dependency trees (among other things). The Universal Dependencies Project distributes its data in the [CoNLL-U format](https://universaldependencies.org/format.html). The code in the next cell reads data in this format from a file-like object:

In [ ]:
ROOT = ('[ROOT]', '[ROOT]', 0)  # Pseudo-word; see comment below

def read_data(source):
    buffer = [ROOT]
    for line in source:
        if not line.startswith('#'):  # Skip lines with comments
            line = line.rstrip()
            if not line:
                yield buffer
                buffer = [ROOT]
            else:
                columns = line.split('\t')
                if columns[0].isdigit():  # Skip range tokens
                    buffer.append((columns[1], columns[3], int(columns[6])))

For this exercise, we will use the training section and the development section of the [English Web Treebank](https://github.com/UniversalDependencies/UD_English-EWT):

In [ ]:
with open('en_ewt-ud-train-projectivized.conllu', encoding='utf-8') as source:
    TRAIN_DATA = list(read_data(source))

with open('en_ewt-ud-dev.conllu', encoding='utf-8') as source:
    DEV_DATA = list(read_data(source))

Both datasets consist of syntactically analysed sentences. In this exercise, an analysed sentence is represented as a list of triples, where the first component of each triple represents a word form, the second component represents the word’s tag, and the third component is an integer specifying the position of the word’s syntactic head, i.e., its parent in the dependency tree. Run the following code cell to see an example:

In [ ]:
DEV_DATA[100]

In this example the head of the word *I* is the word at position&nbsp;2, *ran*; the dependents of *ran* are *I* (position&nbsp;1), *item* (position&nbsp;5), *Internet* (position&nbsp;8), as well as the final punctuation mark. Note that each sentence is preceded by the pseudo-word `[ROOT]` (position&nbsp;0), which represents the structural root of the dependency tree. (It is not dependent on any other word.)

## Step 1: Encoding the data

We provide a function that constructs the word and tag vocabularies from the data:

In [ ]:
PAD, PAD_IDX = '[PAD]', 0
UNK, UNK_IDX = '[UNK]', 1

def make_vocabs(gold_sentences):
    vocab_words = {PAD: PAD_IDX, UNK: UNK_IDX}
    vocab_tags = {PAD: PAD_IDX}
    for sentence in gold_sentences:
        for word, tag, _ in sentence:
            if word not in vocab_words:
                vocab_words[word] = len(vocab_words)
            if tag not in vocab_tags:
                vocab_tags[tag] = len(vocab_tags)
    return vocab_words, vocab_tags

These functions implement the following specification:

**make_vocabs** (*gold_sentences*)

> Returns a pair of two vocabularies, represented as dictionaries: a *word vocabulary* and a *tag vocabulary*. The word vocabulary maps the unique words in the *gold_sentences* to a contiguous range of integers between $0$ and $W+1$, where $W$ is the total number of unique words. Similarly, the tag vocabulary maps the unique part-of-speech tags in the *gold_sentences* to a range between $0$ and $T$, where $T$ is the total number of unique tags. The special words `[PAD]` (used later for padding undefined values) and `[UNK]` (used in place of unknown words at prediction time) are mapped to the indexes&nbsp;0 and&nbsp;1, respectively. The special tag `[PAD]` is mapped to the index&nbsp;0.

In [ ]:
make_vocabs(TRAIN_DATA)[1]

## Step 2: Parser, static part

The parser consists of two parts: a **static part** that implements the logic of the arc-standard transition system, and a **non-static part** that contains the learning component. In this section we cover the static part.

In the arc-standard algorithm, the next move (transition) of the parser is predicted based on features extracted from the current parser configuration, with references to the words and part-of-speech tags of the input sentence. On the Python side of things, we represent parser configurations as triples

$$
(i, \mathit{stack}, \mathit{heads})
$$

where $i$ is an integer specifying the position of the next word in the buffer, $\mathit{stack}$ is a list of integers specifying the positions of the words currently on the stack (with the topmost element last in the list), and $\mathit{heads}$ is a list of integers specifying the positions of the currently assigned head words. To illustrate this representation, the initial configuration for the sample sentence above is

In [ ]:
(0, [], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

and a possible final configuration is

In [ ]:
(10, [0], [0, 2, 0, 5, 5, 2, 8, 8, 2, 2])

**Note:** In the lecture, both the buffer and the stack were presented as list of words. Here we only represent the *stack* as a list of words. To represent the *buffer*, we simply record the position of the next word that has not been processed yet (the integer $i$). This acknowledges the fact that the buffer (in contrast to the stack) can never grow, but will be processed from left to right.

The next cell contains skeleton code for the static part of the parser:

In [ ]:
class ParserCore(object):

    # Parser moves are specified as integers.

    MOVES = tuple(range(3))

    SH, LA, RA = MOVES

    @staticmethod
    def initial_config(num_words):
        return 0, [], [0] * num_words

    @staticmethod
    def valid_moves(config):
        pos, stack, heads = config
        moves = []
        if pos < len(heads):
            moves.append(ParserCore.SH)
        if len(stack) >= 2:
            moves.append(ParserCore.LA)
            moves.append(ParserCore.RA)
        return moves

    @staticmethod
    def next_config(config, move):
        pos, stack, heads = config
        stack = list(stack)  # copy because we will modify it
        if move == ParserCore.SH:
            stack.append(pos)
            pos += 1
        else:
            heads = list(heads)  # copy because we will modify it
            s1 = stack.pop()
            s2 = stack.pop()
            if move == ParserCore.LA:
                heads[s2] = s1
                stack.append(s1)
            if move == ParserCore.RA:
                heads[s1] = s2
                stack.append(s2)
        return pos, stack, heads

    @staticmethod
    def is_final_config(config):
        pos, stack, heads = config
        return pos == len(heads) and len(stack) == 1

Your task is to implement this interface according the following specification:

**initial_config** (*num_words*)

> Returns the initial configuration for a sentence with the specified number of words.

**valid_moves** (*config*)

> Returns the list of valid moves for the specified configuration. Note that moves are represented as integers.

**next_config** (*config*, *move*)

> Applies the specified move (an integer) to the specified configuration and returns the new configuration.

**is_final_config** (*config*)

> Tests whether the specified configuration is a final configuration.

### 🤞 Test your code

To test your implementation, you can run the code below. This code creates the initial configuration for the example sentence, simulates a sequence of moves, and then checks that the resulting configuration is the expected final configuration.

In [ ]:
def test_parser_core():
    moves = [0, 0, 0, 1, 0, 0, 0, 1, 1, 2, 0, 0, 0, 1, 1, 2, 0, 2, 2]
    parser = ParserCore()
    config = parser.initial_config(len(DEV_DATA[100]))
    for move in moves:
        assert move in parser.valid_moves(config)
        config = parser.next_config(config, move)
    assert parser.is_final_config(config)
    assert config == (10, [0], [0, 2, 0, 5, 5, 2, 8, 8, 2, 2])

In [ ]:
test_parser_core()

## Step 3: Oracle

The heart of the non-static part of the parser is the *next move classifier*. To train this classifier, we need training examples of the form $(\mathbf{x}, m)$, where $\mathbf{x}$ is a feature vector extracted from a given parser configuration $c$, and $m$ is the corresponding gold-standard move. To obtain $m$, we need an **oracle**.

Recall that, in the context of transition-based dependency parsing, an oracle is a function that translates a gold-standard dependency tree (here represented as a list of head ids) into a sequence of moves such that, when the parser takes the moves starting from the initial configuration, then it recreates the original dependency tree. Here we ask you to implement the static oracle that was presented in the video lectures.

In [ ]:
def oracle_moves(gold_heads):
    remaining_count = [0] * len(gold_heads)
    for node in gold_heads:
        remaining_count[node] += 1

    config = ParserCore.initial_config(len(gold_heads))
    while not ParserCore.is_final_config(config):
        pos, stack, heads = config
        if len(stack) >= 2:
            s1 = stack[-1]
            s2 = stack[-2]
            if gold_heads[s2] == s1 and remaining_count[s2] == 0:
                move = ParserCore.LA
                yield config, move
                config = ParserCore.next_config(config, move)
                remaining_count[s1] -= 1
                continue
            if gold_heads[s1] == s2 and remaining_count[s1] == 0:
                move = ParserCore.RA
                yield config, move
                config = ParserCore.next_config(config, move)
                remaining_count[s2] -= 1
                continue
        move = ParserCore.SH
        yield config, move
        config = ParserCore.next_config(config, move)

Your implementation should conform to the following specification:

**oracle_moves** (*gold_heads*)

> Translates a gold-standard head assignment for a sentence-specific head-assignment (*gold_heads*) into the corresponding stream of oracle moves. More specifically, this yields pairs $(c, m)$ where $m$ is a move and $c$ is the configuration in which $m$ was taken.

### 🤞 Test your code

You can test your oracle by executing the cell below. This extracts the oracle move sequence from the example sentence and compares it to the gold-standard move sequence `gold_moves`.

In [ ]:
def test_oracle():
    gold_heads = [h for w, t, h in DEV_DATA[100]]
    gold_moves = [0, 0, 0, 1, 0, 0, 0, 1, 1, 2, 0, 0, 0, 1, 1, 2, 0, 2, 2]
    assert list(m for _, m in oracle_moves(gold_heads)) == gold_moves

In [ ]:
test_oracle()

## Step 4: Next move classifier

The next move classifier is implemented by a feedforward network. The input to this network is a vector of integers representing words and tags, as described in the article by [Chen and Manning (2014)](https://www.aclweb.org/anthology/D14-1082/). Here we will use a simple feature model would look at the next word in the buffer and the topmost two words on the stack (for details, see below). The network processes this input as follows:

1. embed the words and tags and concatenate the resulting embeddings
2. send the concatenated embeddings through a linear layer followed by a ReLU
3. pass the output of the non-linearity into a final softmax layer

Your task is to implement the next move classifier based on this specification. Skeleton code for this is available in the next cell. The following choices are reasonable defaults for the hyperparameters of the network architecture:

* width of the word embedding: 50
* width of the tag embedding: 10
* size of the hidden layer: 180

In [ ]:
import torch.nn as nn

class NextMoveClassifier(nn.Module):

    def __init__(self, num_words, num_tokens, word_dim=50, tag_dim=10, hidden_dim=180):
        super().__init__()
        self.word_embedding = nn.Embedding(num_words, word_dim)
        self.tag_embedding = nn.Embedding(num_words, tag_dim)
        self.pipe = nn.Sequential(
            nn.Linear(3 * word_dim + 3 * tag_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 3)
        )

    def forward(self, x):
        embedded = [
            self.word_embedding(x[..., 0]),
            self.word_embedding(x[..., 1]),
            self.word_embedding(x[..., 2]),
            self.tag_embedding(x[..., 3]),
            self.tag_embedding(x[..., 4]),
            self.tag_embedding(x[..., 5]),
        ]
        return self.pipe(torch.cat(embedded, -1))

## Step 5: Parser, non-static part

The next cell contains skeleton code for the non-static part of the parser. We ask you to implement a fixed-window model with the following features:

0. word form of the next word in the buffer
1. word form of the topmost word on the stack
2. word form of the second-topmost word on the stack
3. part-of-speech tag of the next word in the buffer
4. part-of-speech tag of the topmost word on the stack
5. part-of-speech tag of the second-topmost word on the stack

Whenever the value of a feature is undefined, you should use the special value `PAD_IDX`.

In [ ]:
class Parser(ParserCore):

    def __init__(self, vocab_words, vocab_tags):
        self.vocab_words = vocab_words
        self.vocab_tags = vocab_tags
        self.model = NextMoveClassifier(len(vocab_words), len(vocab_tags))

    def featurize(self, word_indices, tag_indices, config):
        i, stack, heads = config
        x = torch.zeros(6, dtype=torch.long)
        x[0] = word_indices[i] if i < len(word_indices) else PAD_IDX
        x[1] = word_indices[stack[-1]] if len(stack) >= 1 else PAD_IDX
        x[2] = word_indices[stack[-2]] if len(stack) >= 2 else PAD_IDX
        x[3] = tag_indices[i] if i < len(tag_indices) else PAD_IDX
        x[4] = tag_indices[stack[-1]] if len(stack) >= 1 else PAD_IDX
        x[5] = tag_indices[stack[-2]] if len(stack) >= 2 else PAD_IDX
        return x
    
    def predict(self, words, tags):
        words = [self.vocab_words.get(w, UNK_IDX) for w in words]
        tags = [self.vocab_tags.get(t, UNK_IDX) for t in tags]
        config = self.initial_config(len(words))
        valid_moves = self.valid_moves(config)
        while valid_moves:
            features = self.featurize(words, tags, config)
            with torch.no_grad():
                scores = self.model.forward(features)

            # We may only predict valid transitions
            best_score, pred_move = float('-inf'), None
            for move in valid_moves:
                if scores[move] > best_score:
                    best_score, pred_move = scores[move], move

            config = self.next_config(config, pred_move)
            valid_moves = self.valid_moves(config)
        i, stack, pred_heads = config
        return pred_heads

Your implementation should comply with the following specification:

**vocab_words**

> The word vocabulary of this parser.

**vocab_tags**

> The tag vocabulary of this parser.

**__init__** (*self*, *vocab_words*, *vocab_tags*)

> Creates a new parser, including the next move classifier that you implemented in Step&nbsp;3. The arguments *vocab_words* and *vocab_tags* are the dictionaries that you created in Step&nbsp;1.

**featurize** (*self*, *word_indices*, *tag_indices*, *config*)

> Returns the input vector to the next move classifier for the specified parser configuration. For the feature model described above, this will be a vector of length&nbsp;6. The *word_indices* and *tag_indices* are the vocabulary indices of the words and part-of-speech tags for the current input sentence, and *config* is a parser configuration as in Step&nbsp;2.

**predict** (*self*, *words*, *tags*)

> Predicts the list of all heads for the specified input sentence. The input sentence is specified in terms of the list of its *words* and the list of its *tags*. This method runs the arc-standard parsing algorithm, at each step asking the next move classifier for the next transition to take. More specifically, the parser uses the highest-scoring transition that is valid in the current configuration.

## Step 6: Next move dataset

The following cell contains skeleton code for a class `NextMoveDataset` that holds the training samples of the next move classifier.

In [ ]:
from torch.utils.data import Dataset

class NextMoveDataset(Dataset):

    def __init__(self, gold_sentences, parser):
        self.xs = []
        self.ys = []
        for parsed_sentence in gold_sentences:
            # Separate the words, gold tags, and gold heads
            words, tags, gold_heads = zip(*parsed_sentence)

            # Encode words and tags using the vocabularies
            words = [parser.vocab_words.get(w, UNK_IDX) for w in words]
            tags = [parser.vocab_tags[t] for t in tags]

            # Call the oracle
            for config, gold_move in oracle_moves(gold_heads):
                self.xs.append(parser.featurize(words, tags, config))
                self.ys.append(gold_move)

    def __len__(self):
        return len(self.xs)

    def __getitem__(self, idx):
        return self.xs[idx], self.ys[idx]

This class conforms to the following specification:

**NextMoveDataset** (*gold_sentences*, *parser*)

> Constructs a PyTorch dataset for the next move classifier. The dataset is constructed by calling *oracle_moves* on each gold-standard sentence in *gold_sentences*, and applying the `featurize` method to the resulting configurations to obtain feature vectors.

## Step 7: Training loop

The last piece of the implementation of the baseline parser is the training loop. This should hold no surprises.

In [ ]:
import torch

import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from tqdm import tqdm

def train(train_data, n_epochs=1, batch_size=100, lr=1e-2):
    # Create the vocabularies
    vocab_words, vocab_tags = make_vocabs(train_data)

    # Instantiate the parser
    parser = Parser(vocab_words, vocab_tags)

    # Create the next move dataset
    dataset = NextMoveDataset(train_data, parser)

    # Instantiate the data loader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Instantiate the optimizer
    optimizer = optim.Adam(parser.model.parameters(), lr=lr)

    # Training loop
    for epoch in range(n_epochs):
        running_loss = 0
        n_examples = 0
        with tqdm(total=sum(2*len(s)-1 for s in train_data)) as pbar:
            for bx, by in dataloader:
                optimizer.zero_grad()
                output = parser.model.forward(bx)
                loss = F.cross_entropy(output, by)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                n_examples += 1
                pbar.set_postfix(loss=running_loss/n_examples)
                pbar.update(len(bx))

    return parser                                            

In [ ]:
PARSER = train(TRAIN_DATA, n_epochs=1)

## Step 8: Evaluation

For evaluation, we use **unlabelled attachment score (UAS)**, which is defined as the percentage of all tokens to which the parser assigns the correct head (as per the gold standard). Note that the calculation excludes the pseudoword at position&nbsp;0 in each sentence.

In [ ]:
def uas(parser, gold_sentences):
    total = 0
    correct = 0
    for sentence in gold_sentences:
        words, tags, gold_heads = zip(*sentence)
        pred_heads = parser.predict(words, tags)
        for gold, pred in zip(gold_heads[1:], pred_heads[1:]):
            total += 1
            correct += int(gold == pred)
    return correct / total

Run the following cell to test the parser.

In [ ]:
print('{:.4f}'.format(uas(PARSER, DEV_DATA)))

For a parser trained for one epoch, you should expect to see an UAS of around 70%.

That&rsquo;s all, folks!